<font size=4 fonc="仿宋" color=white>第1步：导入模块与设置环境</font>

In [2]:
%%time
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential, layers, utils
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Wall time: 0 ns


<font size=4 fonc="仿宋" color=white>第2步：加载数据</font>

In [3]:
%%time
data = pd.read_csv("bike_data.csv", encoding="gbk")
data.head()

Wall time: 103 ms


,OrderNumber,LineNumber,Model
0,cumid51178,1,山地英骑
1,cumid51178,2,山地车水壶架
2,cumid51178,3,运动水壶
3,cumid51184,1,山地英骑
4,cumid51184,2,hl山地外胎


<font size=4 fonc="仿宋" color=white>第3步：数据清洗</font>

In [4]:
data.Model.unique().tolist()

['山地英骑',
 '山地车水壶架',
 '运动水壶',
 'hl山地外胎',
 '山地车内胎',
 '运动型头盔',
 '普通公路车',
 '公路车内胎',
 'hl公路外胎',
 '竞速公路车',
 '公路车水壶架',
 '长袖骑车衣',
 '山地车挡泥板',
 '自行车帽',
 '山地车400',
 'ml山地外胎',
 '修补工具',
 '山地车500',
 '公路车550',
 '短袖经典车衣',
 '旅游型自行车(大)',
 '竞速袜',
 '半掌手套',
 '公路车350',
 'ml公路外胎',
 '水壶包',
 '旅游型自行车(小)',
 '旅游型自行车(中)',
 '旅游车内胎',
 'll公路车外胎',
 '旅游自行车外胎(通用)',
 '万能自行车座',
 '洗车喷剂',
 '经典背心',
 'll山地胎',
 '故障栓钩',
 "Women's Mountain Shorts"]

In [5]:
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace("Women's Mountain Shorts", '未知名称'))

In [6]:
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace("(", ""))
data['Model'] = data['Model'].astype(str).apply(lambda x: x.replace(")", ""))

In [7]:
data['Model'] = data['Model'].astype(str).apply(lambda x: x + " ")
data.head()

,OrderNumber,LineNumber,Model
0,cumid51178,1,山地英骑
1,cumid51178,2,山地车水壶架
2,cumid51178,3,运动水壶
3,cumid51184,1,山地英骑
4,cumid51184,2,hl山地外胎


<font size=4 fonc="仿宋" color=white>第4步：合并数据集，形成长文本</font>

In [8]:
%%time
df = data.groupby(['OrderNumber'])['Model'].sum().reset_index()
df['Model'] = df['Model'].astype(str).apply(lambda x: x.strip())
df.head()

Wall time: 4.51 s


,OrderNumber,Model
0,cumid51176,普通公路车 公路车水壶架
1,cumid51177,运动型头盔 旅游型自行车中
2,cumid51178,山地英骑 山地车水壶架 运动水壶
3,cumid51179,万能自行车座 公路车内胎 hl公路外胎 普通公路车
4,cumid51180,普通公路车 公路车水壶架 运动水壶 运动型头盔 长袖骑车衣


In [9]:
texts = df.Model.tolist()

In [10]:
texts[:5]

['普通公路车 公路车水壶架',
 '运动型头盔 旅游型自行车中',
 '山地英骑 山地车水壶架 运动水壶',
 '万能自行车座 公路车内胎 hl公路外胎 普通公路车',
 '普通公路车 公路车水壶架 运动水壶 运动型头盔 长袖骑车衣']

In [11]:
all_texts = texts[0]
for i in texts[1:]:
    all_texts += " " + i

In [12]:
all_texts = list(set(all_texts.split(' ')))
all_texts.append("<PAD>")
all_texts = set(all_texts)
all_texts

{'<PAD>',
 'hl公路外胎',
 'hl山地外胎',
 'll公路车外胎',
 'll山地胎',
 'ml公路外胎',
 'ml山地外胎',
 '万能自行车座',
 '修补工具',
 '公路车350',
 '公路车550',
 '公路车内胎',
 '公路车水壶架',
 '半掌手套',
 '山地英骑',
 '山地车400',
 '山地车500',
 '山地车内胎',
 '山地车挡泥板',
 '山地车水壶架',
 '故障栓钩',
 '旅游型自行车中',
 '旅游型自行车大',
 '旅游型自行车小',
 '旅游自行车外胎通用',
 '旅游车内胎',
 '普通公路车',
 '未知名称',
 '水壶包',
 '洗车喷剂',
 '短袖经典车衣',
 '竞速公路车',
 '竞速袜',
 '经典背心',
 '自行车帽',
 '运动型头盔',
 '运动水壶',
 '长袖骑车衣'}

In [13]:
len(all_texts)

38

In [14]:
text_to_id = {c: i for i, c in enumerate(all_texts)}
id_to_text = {i: c for i, c in enumerate(all_texts)}

In [15]:
def add_string(string):
    if len(string.split(" "))==1:
        return "<PAD> <PAD> <PAD> " + string
    else:
        return "<PAD> <PAD> " + string


df['Model'] = df['Model'].astype(str).apply(add_string)
df.head()

,OrderNumber,Model
0,cumid51176,<PAD> <PAD> 普通公路车 公路车水壶架
1,cumid51177,<PAD> <PAD> 运动型头盔 旅游型自行车中
2,cumid51178,<PAD> <PAD> 山地英骑 山地车水壶架 运动水壶
3,cumid51179,<PAD> <PAD> 万能自行车座 公路车内胎 hl公路外胎 普通公路车
4,cumid51180,<PAD> <PAD> 普通公路车 公路车水壶架 运动水壶 运动型头盔 长袖骑车衣


In [16]:
df['Model'] = df['Model'].astype(str).apply(lambda x: x.split(' '))
df.head()

,OrderNumber,Model
0,cumid51176,"[<PAD>, <PAD>, 普通公路车, 公路车水壶架]"
1,cumid51177,"[<PAD>, <PAD>, 运动型头盔, 旅游型自行车中]"
2,cumid51178,"[<PAD>, <PAD>, 山地英骑, 山地车水壶架, 运动水壶]"
3,cumid51179,"[<PAD>, <PAD>, 万能自行车座, 公路车内胎, hl公路外胎, 普通公路车]"
4,cumid51180,"[<PAD>, <PAD>, 普通公路车, 公路车水壶架, 运动水壶, 运动型头盔, 长袖骑车衣]"


In [17]:
x = []
y = []

for values in df['Model'].tolist():
    if len(values) == 4:
        x.append(" ".join(values[:3]))
        y.append(values[-1])
    else:
        for i in np.arange(0, len(values)-3, step=1):
            x.append(" ".join(values[i: i+3]))
            y.append("".join(values[i+3]))

In [18]:
texts = []
for text in x:
    texts.append([text_to_id[i] for i in text.split(' ')])

label = [text_to_id[i] for i in y]
texts[:5]

[[37, 37, 12], [37, 37, 30], [37, 37, 9], [37, 9, 25], [37, 37, 3]]

In [19]:
len(texts)

35503

In [20]:
texts = np.array(texts)
label = np.array(label)

In [21]:
texts = texts / len(all_texts)

In [22]:
x = texts.reshape(texts.shape[0], 3, 1)
x.shape

(35503, 3, 1)

In [23]:
y = tf.keras.utils.to_categorical(label)
y.shape

(35503, 37)

<font size=4 fonc="仿宋" color=white>第5步：划分训练集与测试集</font>

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, 
                                                    random_state=2021)

<font size=4 fonc="仿宋" color=white>第6步：构建模型</font>

In [25]:
model = Sequential([
    layers.LSTM(units=800, input_shape=(3, 1), return_sequences=True),
    layers.Dropout(0.3),
    layers.LSTM(units=800, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(units=800),
    layers.Dropout(0.2),
    layers.Dense(y.shape[1], activation='softmax')
])

In [26]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 800)            2566400   
_________________________________________________________________
dropout (Dropout)            (None, 3, 800)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 800)            5123200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 800)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 800)               5123200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 37)                2

In [ ]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=500, 
          batch_size=64)

Epoch 1/500
444/444 [==============================] - 416s 912ms/step - loss: 3.0690 - accuracy: 0.1763 - val_loss: 2.9468 - val_accuracy: 0.1810
Epoch 2/500
444/444 [==============================] - 388s 873ms/step - loss: 2.8414 - accuracy: 0.1953 - val_loss: 2.7676 - val_accuracy: 0.1984
Epoch 3/500
444/444 [==============================] - 365s 822ms/step - loss: 2.6870 - accuracy: 0.2150 - val_loss: 2.5723 - val_accuracy: 0.2346
Epoch 4/500
444/444 [==============================] - 329s 740ms/step - loss: 2.5604 - accuracy: 0.2430 - val_loss: 2.5781 - val_accuracy: 0.2728
Epoch 5/500
444/444 [==============================] - 325s 731ms/step - loss: 2.4916 - accuracy: 0.2578 - val_loss: 2.3964 - val_accuracy: 0.2936
Epoch 6/500
444/444 [==============================] - 318s 716ms/step - loss: 2.4403 - accuracy: 0.2692 - val_loss: 2.3989 - val_accuracy: 0.2915
Epoch 7/500
444/444 [==============================] - 319s 718ms/step - loss: 2.3910 - accuracy: 0.2785 - val_loss: 2